<a href="https://colab.research.google.com/github/TwoSoul/TLC_demo/blob/master/TLC_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Жёлтое такси в Нью-Йорке**
### Была поставлена задача проекта — научиться предсказывать количество поездок в ближайшие часы в каждом районе Нью-Йорка; для простоты мы определили прямоугольные регионы. Для того, чтобы её решить, сырые данные были агрегированы по часам и регионам. 
### Для предсказывания временных рядов, были построенны шесть моделей регрессии CatBoost для предсказывания на час вперед, на 2, 3, 4, 5 и 6 соответственно. Для построения моделей использовались данные с 2016-01-16 по 2016-06-30, были построенны следующие признаки:
* id региона
* номер кластера
* месяц, день, час, день недели, признак выходного дня
* кол-во поездок в моменты времени T, T-1, T-2, T-3, T-4, T-5, T-24, T-48, T-168 и T-336
* 21 гармоника (синусы косинусы)
* суммарное количество поездок за предшествующие полдня, сутки
Попробуем добавить следующие признаки:
* кол-в поездок в регион из других регионов
* cреднее кол-во поездок в определенный час определенного дня недели
* средняя длительность поездки в секунда за час, два часа
* средняя стоимость поездки за час, два часа
* среднее кол-во пассажиров за час, два часа
* средняя дистанция поездки за час, два часа
* признак рабочего дня
### Модели были обучены на данных с 2016-01-16 по 2016-05-31, для оценки использовались данные с 2016-06-01 по 2016-06-30, ошибка MAE = 20,496.
## ***Для просмотра демонстрации необходимо последовательно запустить нижние три ячейки.***



In [ ]:
#@title Инициализация данных
# загрузка координат регионов
!gdown https://drive.google.com/uc?id=1ZIkQ5-8_CB3auXCPemf4O2xPra4WYMD5
# загрузка набора данных
!gdown https://drive.google.com/uc?id=1a1sw2zZJI_jdjNsT1PVVBitQe8PnTaFq

import numpy as np
import pandas as pd

from ipywidgets import widgets
from IPython.display import clear_output

import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

data_show = pd.read_csv('data_show.csv', index_col=0, parse_dates=['datetime_pickup'])
#data_list = np.sort(data_show[data_show['datetime_pickup'] >= datetime.datetime(2016, 6, 1)].datetime_pickup.unique())

# получим границы всех регионов
regions = pd.read_csv('regions.csv', sep=';')
# 102 региона
region_list = ['1077', '1181', '1182', '1184', '1279', '1280', '1287', '1331', 
                '1332', '1333', '1335', '1336', '1337', '1383', '1384', '1385', 
                '1386', '1434', '1075', '1076', '1125', '1126', '1127', '1128', 
                '1129', '1130', '1131', '1176', '1178', '1179', '1180', '1229', 
                '1230', '1231', '1232', '1233', '1234', '1235', '1281', '1282', 
                '1283', '1284', '1285', '1286', '1334', '1338', '1382', '1684', 
                '1734', '1783', '2068', '2069', '2118', '2119', '2168', '1132', 
                '1177', '1221', '1222', '1227', '1228', '1272', '1274', '1326',
                '1327', '1376', '1377', '1378', '1426', '1482', '1532', '1533', 
                '1630', '1172', '1173', '1174', '1175', '1183', '1223', '1224', 
                '1225', '1273', '1278', '1339', '1380', '1387', '1388', '1389', 
                '1390', '1431', '1435', '1436', '1437', '1438', '1439', '1441', 
                '1442', '1480', '1483', '1530', '1580', '1733']
regions = regions[regions['region'].isin(region_list)]
# задаем координаты полигонов регионов
regions['x1'] = [[round(row[1], 6), round(row[4], 6)] for row in regions.to_numpy()]
regions['x2'] = [[round(row[2], 6), round(row[4], 6)] for row in regions.to_numpy()]
regions['x3'] = [[round(row[2], 6), round(row[3], 6)] for row in regions.to_numpy()]
regions['x4'] = [[round(row[1], 6), round(row[3], 6)] for row in regions.to_numpy()]
regions['x5'] = [[round(row[1], 6), round(row[4], 6)] for row in regions.to_numpy()]

# зададим регионы в формате GeoJson
def get_regins_geojson(regions):
    features = []
    for row in regions[['region', 'x1', 'x2', 'x3', 'x4', 'x5']].values:
        features.append({
            "properties": {"name": row[0]},
            "id": row[0],
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [row[1:].tolist()]
            }
        })
    return {
        "type": "FeatureCollection",
        "features": features
    }

regions_geojson = get_regins_geojson(regions)

coord_empire_state_building = [40.748306, -73.985756]

clear_output()
print("Данные инициализированны!")

Данные инициализированны!


In [ ]:
#@title Карты с визуализацией реального и прогнозируемого спроса на такси в выбираемый момент времени. Для построения карт необходимо указать момент времени T и час h для построения предсказания в момент времени T+h. При наведении на регион показывает его номер и значение. На правой карте изображены регионы с исходными данными, слева с предсказаниями. { run: "auto" }
T_date = '2016-06-27' #@param {type:"date", min:'2016-06-01', max:'2016-06-30'}
T_time = 8 #@param {type:"slider", min:0, max:23}
H_hour = 3 #@param {type:"slider", min:1, max:6}

T_time = str(T_time)
if len(T_time) == 1:
    T_time = '0' + T_time

t = datetime.datetime.strptime(T_date + " %s:00:00" % T_time, "%Y-%m-%d %H:%M:%S")
h = H_hour

data_t_moment = data_show[data_show['datetime_pickup'] == t][['region', 'h' + str(h), str(h) + 'h_pred']]
fig = go.Figure()

fig.add_trace(go.Choroplethmapbox(geojson=regions_geojson, locations=data_t_moment.region,  colorscale = 'aggrnyl', 
                                    zmax=max(data_t_moment['h' + str(h)].max(), data_t_moment[str(h) + 'h_pred'].max()),
                                    z=data_t_moment['h' + str(h)], subplot='mapbox', text=data_t_moment.region, 
                                    hovertemplate='<b>Регион</b>: <b>%{text}</b><br><b>Истинное значение</b>: %{z}<br>'))
fig.add_trace(go.Choroplethmapbox(geojson=regions_geojson, locations=data_t_moment.region, colorscale = 'aggrnyl', 
                                    zmax=max(data_t_moment['h' + str(h)].max(), data_t_moment[str(h) + 'h_pred'].max()),
                                    z=data_t_moment[str(h) + 'h_pred'], subplot='mapbox2', text=data_t_moment.region, 
                                    hovertemplate='<b>Регион</b>: <b>%{text}</b><br><b>Предсказанное значение</b>: %{z}<br>', showscale=False))

fig.update_layout(
    autosize=True,
    hovermode='closest',
    margin={'l': 0, 'r': 0, 't': 0, 'b': 0},
    mapbox=dict(
        style='open-street-map',
        domain={'x': [0, 0.49], 'y': [0, 1]},
        bearing=0,
        center=dict(
            lat=coord_empire_state_building[0],
            lon=coord_empire_state_building[1]
        ),
        pitch=0,
        zoom=10
    ),
    mapbox2=dict(
        style='open-street-map',
        domain={'x': [0.51, 1], 'y': [0, 1]},
        bearing=0,
        center=dict(
            lat=coord_empire_state_building[0],
            lon=coord_empire_state_building[1]
        ),
        pitch=0,
        zoom=10
    ),
)

fig.show()

In [ ]:
#@title Временной ряд фактического и прогнозируемого спроса на такси в выбираемой области. (для ограничения и сдвига области можно использовать слайдер под графиком) { run: "auto" }
region = "1332" #@param [1077, 1181, 1182, 1184, 1279, 1280, 1287, 1331, 1332, 1333, 1335, 1336, 1337, 1383, 1384, 1385, 1386, 1434, 1075, 1076, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1176, 1178, 1179, 1180, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1281, 1282, 1283, 1284, 1285, 1286, 1334, 1338, 1382, 1684, 1734, 1783, 2068, 2069, 2118, 2119, 2168, 1132, 1177, 1221, 1222, 1227, 1228, 1272, 1274, 1326, 1327, 1376, 1377, 1378, 1426, 1482, 1532, 1533, 1630, 1172, 1173, 1174, 1175, 1183, 1223, 1224, 1225, 1273, 1278, 1339, 1380, 1387, 1388, 1389, 1390, 1431, 1435, 1436, 1437, 1438, 1439, 1441, 1442, 1480, 1483, 1530, 1580, 1733] 
hour = "3" #@param [1,2,3,4,5,6]
region = int(region)

data_region = data_show[data_show['region'] == region].sort_values('datetime_pickup')

trace_real = go.Scatter(x=data_region['datetime_pickup'], 
                        y=data_region['h%s' % hour],
                        name = 'Фактические значения'
                       )

trace_predicted = go.Scatter(x=data_region['datetime_pickup'], 
                             y=data_region['%sh_pred' % hour],
                             name = 'Предсказанные значения'
                            )

data = [trace_real, trace_predicted]

layout = dict(title = 'Временной ряд фактического и прогнозируемого спроса на такси в %d регионе на %s ч. вперед' % (region, hour),
              xaxis = dict(rangeslider = dict(visible = True),
                           type = 'date'),
              font = dict(family = 'Balto'),
              height=600,
              margin = dict(l = 20, r = 10, b = 0, t = 50, pad = 0),
             )
timeseries_f = go.FigureWidget(data = data, layout = layout)
timeseries_f.show()